# Defining simulation settings

We start by importing the `openmc` module, which contains the API to generate the input files for OpenMC.

In [1]:
import openmc
openmc.config['cross_sections'] = '/home/student/openmc_ncrystal_vm/endfb-viii.0-hdf5/cross_sections.xml'

Once we defined the materials and geometry for our simulation, we need to indicate to OpenMC the parameters for our simulation. This is done with the class `Settings()`, which generates the file `settings.xml`. This file, along with `materials.xml` and `geometry.xml` forms the minimum set to run OpenMC.

In [2]:
settings = openmc.Settings()

The type of simulation is set with the parameter `.run_mode`. It takes a string with the following values:

* `"eigenvalue"`: computes the multiplication factor and fundamental mode of the associated critical reactor for multiplicative systems.
* `"fixed source"`: transport particles with a fixed source.
* `"volume"`: computes the volumes of cells using Monte Carlo.
* `"plot"`: makes 2D and 3D plots of the geometry.
* `"particle restart"`: restarts a simulation.

In [3]:
settings.run_mode = "eigenvalue"

In the `eigenvalue` and `fixed source` modes we need to indicate the number of particles to simulate with the attributes `.particles` (number of particles per batch) and `.batches` (number of batches). E.g.:

In [4]:
settings.batches = 10
settings.particles = 1000

makes a 10000 particle simulation, divided into 10 batches of 1000 particles.

## Fixed source simulation

To perform a fixed source simulation it is necessary to define an external source. The source is specified in the `.source` parameter of the `Settings()` class. The source is an object of the `Source()` class and has as attributes the distributions in space, angle and energy. E.g. to define an isotropic $1$ MeV neutron source in the origin of coordinates we do:

In [5]:
source = openmc.Source()
source.space = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([1.0e6], [1.0]) # Energy units are MeV

settings = openmc.Settings()
settings.source = source

Distributions are contained in the `openmc.stats` sub module. These include 1D, 2D and 3D distributions that could be used for the different variables of the source:

Space distributions:

* `openmc.stats.Point()`: delta distribution in space, $\delta(\vec{r}-\vec{r}_0)$.
* `openmc.stats.Box()`: uniform distribution over a bos defined with two points: lower left and upper right.
* `openmc.stats.CartesianIndependent()`: 3D distribution factorized in univariate independent distributions for $x$, $y$ and $z$, $D(\vec{r}) = X(x)Y(y)Z(z)$.  

Angular distributions:
* `openmc.stats.Isotropic()`: isotropic distribution in the unit sphere.
* `openmc.stats.Monodirectional()`: delta distriubiton in angle, $\delta(\hat{\Omega}-\hat{\Omega}_0)$.
* `openmc.stats.PolarAzimuthal()`: angle distribution factorized in univariate independent disitrubtions for the cosine of the polar angle $\mu = \cos(\theta)$ and the azimuthal angle $\phi$, respect to a give direction (by default, $+z$).

Univariate distributions (could be used for energy or to define other distributions):
* `openmc.stats.Discrete()`: distribution defined as a set of delta functions.
* `openmc.stats.Uniform()`: uniform distribution on an interval $[a,b]$.
* `openmc.stats.Tabular()`: piecewise continuous distribution defined by a set of points and an interpolation law (histogram, lin-lin, lin-log, log-lin ot log-log).
* `openmc.stats.Watt()`: Watt fission distribution for fission: $\chi(E) = C e^{-E/a} \sinh{\sqrt{b E}}$).
* `openmc.stats.Maxwell()`: Maxwell distribution for fission : $\chi(E) = C \sqrt{E} e^{-E/kT}$. Important note: this is *not* the thermal energy distribution $M(E) = C E e^{-E/kT}$.
* `openmc.stats.Legendre()`: distribution given by an expansion in Legendre polynomials=.
* `openmc.stats.Mixture()`: discrete distribution where the values are other distributions (similar to `FCEL` in MCNP).

As an example, we will calculate the leakage of neutrons from a $10$ cm radius water sphere with a $1$ MeV isotropic source:

In [8]:
# Material definition

h2o = openmc.Material()
h2o.add_nuclide("H1", 2.0, "ao")
h2o.add_nuclide("O16", 1.0, "ao")
h2o.set_density("g/cm3", 1)
h2o.add_s_alpha_beta("c_H_in_H2O")

mats = openmc.Materials([h2o])
mats.export_to_xml()

# Geometry definition

sphere = openmc.Sphere(r=10, boundary_type="vacuum")
cell = openmc.Cell(region=-sphere, fill=h2o)
universe = openmc.Universe(cells=[cell])

geom = openmc.Geometry(universe)
geom.export_to_xml()

# Settings definition

source = openmc.Source()
source.space = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([1.0e6], [1.0])

settings = openmc.Settings()
settings.source = source
settings.run_mode = "fixed source"
settings.batches = 10
settings.particles = 10000
settings.export_to_xml()

After we generated the necessary `xml` files, we can start the run with `openmc.run()`. We can also start the run using the `openmc` command in the terminal.

In [11]:
!rm tallies.xml # delete the tally file since this run does not specify one
!rm statepoint.*.h5
!rm summary.h5
openmc.run()

rm: cannot remove 'statepoint.*.h5': No such file or directory
rm: cannot remove 'summary.h5': No such file or directory
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################    